In [ ]:
# 各種import跟資料設定

import requests
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import time,random
from tqdm.notebook import tqdm as tqdm  #進度條

# 共享雲端，要先add shortcut to drive，操作前先執行
from google.colab import drive
drive.mount('/content/drive')

item_list = ["01","02"]
crop_list = ["C01","C02","C03","C04","C05","C06"]
# crop_list = ["C01"] #  測試用，只跑1個作物
city_list = ["0001","0002","0003","0004","0005","0006","0007","0008","0009","0010","0011","0012","0013","0014","0015","0016","0017","0018","0020","0063","0065","0066"]
# city_list = ["0009","0010"]  # 測試用，只跑2地區
item_dict = {"01":"一期作","02":"二期作"}
crop_dict = {"C01":"梗稻","C02":"硬秈稻","C03":"軟秈稻","C04":"梗糯稻","C05":"秈糯稻","C06":"陸稻"}
city_dict = {"0001":"新北市","0002":"宜蘭縣","0003":"桃園市","0004":"新竹縣","0005":"苗栗縣","0006":"臺中市","0007":"彰化縣","0008":"南投縣","0009":"雲林縣","0010":"嘉義縣","0011":"臺南市","0012":"高雄市","0013":"屏東縣","0014":"臺東縣","0015":"花蓮縣","0016":"澎湖縣","0017":"基隆市","0018":"新竹市","0020":"嘉義市","0063":"臺北市","0065":"金門縣","0066":"連江縣"}
start_year = 100  # 初始年
end_year = 110  # 截止年
nodata = [] #沒抓到資料的縣市鄉鎮

Mounted at /content/drive


In [ ]:
total_progress = len(item_list)*len(crop_list)*len(city_list)*(end_year-start_year+1) #進度條分母：22縣市*6種作物*2種期作*年分，如果只爬一年=264個
progress = tqdm(total=total_progress)  


for each_accountingyear in range(end_year,start_year-1,-1):   
  for each_item in item_list:
    for each_crop in crop_list:
      for each_city in city_list:

        # 參數
        accountingyear = each_accountingyear
        item = each_item
        crop = each_crop
        city = each_city

        # 執行POST REQUEST
        url = "https://agr.afa.gov.tw/afa/pgricetown.jsp"
        myobj = {
              "accountingyear": accountingyear,
              "item": item,
              "crop": crop,
              "city": city,
        }
        
        re = requests.post(url, data=myobj)
        soup = BeautifulSoup(re.text, "html.parser")
        
        all_tables = soup.find("div", {"class": "DivRestTbl"})  #  整個表格
        
        # 抓取欄位名稱
        columns = [td.text.replace("\n", "") for td in all_tables.find("tr").find_all("td")]
        # print(columns)  #TESTING

        # 抓取欄位單位，但沒用到
        # units_all = all_tables.find_all("tr")[1]
        # units = [td.text.replace("\n", "") for td in units_all.find_all("td")]
        # # print(units)  #TESTING

        # 抓取每個鄉鎮對應資料內容，並取消千分位
        trs = all_tables.find_all("tr")[2:-1]   #這邊改成2:-1看看??
        rows = []
        for tr in trs:
          rows.append([td.text.replace("\n", "").replace(",","") for td in tr.find_all("td")])
        # print(rows)  # TESTING
        
        
        #利用pandas製作Dataframe
        try:
          df = pd.DataFrame(data=rows, columns=columns)
          # df.head()  # 測試用，print出前五行
          # 要解決自動新增資料夾
          df.insert(0,'作物代號',crop)
          df.insert(0,'作物',crop_dict[crop])
          df.insert(0,'期作別代號',item)
          df.insert(0,'期作別',item_dict[item])
          df.insert(0,'年份',accountingyear)
          df.insert(0,'縣市',city_dict[city])
          df.to_csv(f'/content/drive/MyDrive/ccClubProject/csv/110/1212_{accountingyear}_{item}_{crop}_{city}.csv')   #進行測試中要註解，先不寫入；如果error噴FileNotFoundError請確認是否開過雲端權限
          print(f"{accountingyear}_{item}_{crop}_{city} 抓取完畢")
          
        except ValueError:
          nodata.append(f'{accountingyear}_{item}_{crop}_{city}')  #沒抓到資料的list
          print(f'{accountingyear}_{item}_{crop}_{city} not successful(No such data)') 
        
        progress.update(1)  # 進度條手動增加1

        '''
        # 測試為何期作別代號匯到csv會是1不是01 (未果)
        print(item)
        print(item_dict)
        print(item_list)
        '''

        # 暫停3~7秒之間，避免存取過於頻繁
        # time.sleep(random.randint(3,7)) 

In [ ]:
# 印出查無資料的request 目前要手動新增到記事本
print(f"總共有{len(nodata)}個檔案查無資料，清單放入list如下：")
print(nodata)  

In [ ]:
# 把個別檔案合成當年度大檔
files = glob('drive/MyDrive/ccClubProject/csv/110/*.csv')
print(len(files))  #測試是否正確抓到

# 僅讀取特定欄位時：usecols=['name','name']，默認情況不指定該參數表示選取所有欄位
df = pd.concat((pd.read_csv(file, dtype={'縣市': str, '年份': str, '期作別代號': str, '期作別代號': str, '作物': str, '作物代號': str, '縣市鄉鎮名稱': str, '初步種植面積':float, '實際種植面積':float, '收穫面積':float, '無收穫面積':float, '稻穀總產量':float, '稻穀單位產量':float, '糙米總產量':float, '糙米單位產量':float}) for file in files), ignore_index=True)
 
df.to_csv(f'/content/drive/MyDrive/ccClubProject/csv/110年度all.csv') 

In [ ]:
# 做好的當年度大檔或是87-110年度的大檔串接到縣市鄉鎮代號

df1 = pd.read_csv(f'/content/drive/MyDrive/ccClubProject/csv/100_110年度all.csv')

df2 = pd.read_excel(f'/content/drive/MyDrive/ccClubProject/TWM368towncode.xlsx')

# 農情報告資源網的縣市鄉鎮名稱用的是"台"，但想要用"臺"去連
dff = df1.merge(df2, on='縣市鄉鎮名稱',left_index=False, right_index=False, sort=False)   

dff.to_csv(f'/content/drive/MyDrive/ccClubProject/csv/test.csv') 